In [ ]:
# @title Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Tuned Text-Embeddings on Vertex AI

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/tuned_text-embeddings.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo">Open in Colab
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/generative_ai/tuned_text-embeddings.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/tuned_text-embeddings.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

This notebook will walk you through the process of fine-tuning text-embeddings. By adapting a text-embedding model to your specific domain or task, you can achieve better results. See also [tuning text-embeddings](https://cloud.google.com/vertex-ai/generative-ai/docs/models/tune-embeddings).

### Objective

In this tutorial, you learn how to tune a text-embedding model, textembedding-gecko.

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI
- Cloud Storage

The steps include:

- Run a text-embedding model tuning job on Vertex AI Pipelines.
- Deploy your tuned text-embedding model to an endpoint.
- Examine the quality metrics to assess your tuned model.
- Get embeddings from your tuned model for downstream tasks.

### Dataset

This notebook uses a synthetic dataset (corpus, queries, and labels) from [Alphabet's annual financial performance report (10K form)](https://abc.xyz/assets/ff/7c/06d6f493f6462caf08e8502ffc33/596de1b094c32cf0592a08edfe84ae74.html).

* [corpus.jsonl](https://storage.googleapis.com/cloud-samples-data/ai-platform/embedding/goog-10k-2024/r11/corpus.jsonl) (397Ki),
* [queries.jsonl](https://storage.googleapis.com/cloud-samples-data/ai-platform/embedding/goog-10k-2024/r11/queries.jsonl) (321Ki)
* [test.tsv](https://storage.googleapis.com/cloud-samples-data/ai-platform/embedding/goog-10k-2024/r11/test.tsv) (3.7Ki)
* [train.tsv](https://storage.googleapis.com/cloud-samples-data/ai-platform/embedding/goog-10k-2024/r11/train.tsv) (29Ki)
* [validation.tsv](https://storage.googleapis.com/cloud-samples-data/ai-platform/embedding/goog-10k-2024/r11/validation.tsv) (3.7Ki)

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and [Cloud Storage pricing](https://cloud.google.com/storage/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

This tutorial requires you to install the `google-cloud-aiplatform` package.

In [ ]:
! pip3 install --upgrade --user --quiet google-cloud-aiplatform

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable APIs](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,documentai.googleapis.com).

4. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

See also [setting up permissions & resources for tuning text-embedding models](https://cloud.google.com/vertex-ai/generative-ai/docs/models/tune-embeddings#project-setup).

### Initialize Vertex AI Platform

Import and Initialize AI platform for your project and region.

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See also [Locate the project ID](https://support.google.com/googleapi/answer/7014113).

See also [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
# @title (Required) Set PROJECT_ID and REGION
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}
if not PROJECT_ID.strip():
    raise ValueError("'PROJECT_ID' is required.")
if not REGION.strip():
    raise ValueError("'REGION' is required.")
!gcloud config set project {PROJECT_ID}

import vertexai
from google.cloud.aiplatform import pipeline_jobs
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel

vertexai.init(project=PROJECT_ID, location=REGION)

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Colab:**

In [ ]:
# @title (Required on Colab) `authenticate_user()`
import builtins
import os
import sys

running_in_colab = "google.colab" in sys.modules and hasattr(builtins, "get_ipython")
if running_in_colab and not os.getenv("IS_TESTING"):
    from google.colab import auth

    auth.authenticate_user(project_id=PROJECT_ID)

**2. Vertex AI Workbench**

Make sure that [the Compute Engine default service account](https://cloud.google.com/compute/docs/access/service-accounts#default_service_account) running a Vertex AI workbench instance has the permission iam.serviceAccounts.actAs (most likely through [roles/iam.serviceAccountUser](https://cloud.google.com/iam/docs/understanding-roles#iam.serviceAccountUser)) at [the IAM & Admin page of the Cloud Console](https://console.cloud.google.com/iam-admin). This permission allows a workbench instance to act as the service account when interacting with other Google Cloud services.

**3. Local JupyterLab instance, uncomment and run:**

In [ ]:
# !gcloud auth login

## Tune Text-Embeddings

(Optionally), to resume this tutorial from where you left off with your previous tuning session, set **`TUNING_JOB_ID`** accordingly. Or, clear **`TUNING_JOB_ID`** to start over a fresh tuning session.

This tutorial creates a tuning job of a Vertex AI pipeline for tuning a text-embedding model within your project. See also [creating tuning jobs with parameters and defaults](https://cloud.google.com/vertex-ai/generative-ai/docs/models/tune-embeddings#create-embedding-tuning-job), the latest [text-embedding models and eligible tasks](https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-text-embeddings#api_changes_to_models_released_on_or_after_august_2023).

In [ ]:
# @title (Optional) Set `TUNING_JOB_ID` to resume an existing tuning session: { run: "auto" }
TUNING_JOB_ID = ""  # @param {type: "string", placeholder:"(Optional) Enter the name of an existing Vertex Pipelines job."}

In [ ]:
# @title (Required) Resume an existing or start a fresh tuning session (depending on TUNING_JOB_ID)
# @markdown ### Pipeline Parameters
# @markdown `queries_path`: The URI of the Cloud Storage bucket storing query data, starting with `gs://`.
queries_path = "gs://cloud-samples-data/ai-platform/embedding/goog-10k-2024/r11/queries.jsonl"  # @param {type: "string"}
# @markdown ---
# @markdown `corpus_path`: The Cloud Storage URI for the corpus data, starting with `gs://`.
corpus_path = "gs://cloud-samples-data/ai-platform/embedding/goog-10k-2024/r11/corpus.jsonl"  # @param {type: "string"}
# @markdown ---
# @markdown `train_label_path`: The Cloud Storage URI of the train label data location, starting with `gs://`.
train_label_path = "gs://cloud-samples-data/ai-platform/embedding/goog-10k-2024/r11/train.tsv"  # @param{type: "string"}
# @markdown ---
# @markdown `validation_label_path`: Optional. The Cloud Storage URI of the validation label data location, starting with `gs://`. Passing an empty string will direct the pipeline to autosplit the validation dataset from the training dataset.
validation_label_path = "gs://cloud-samples-data/ai-platform/embedding/goog-10k-2024/r11/validation.tsv"  # @param{type: "string"}
# @markdown ---
# @markdown `test_label_path`: Optional. The Cloud Storage URI of the test label data location, starting with `gs://`. Passing an empty string will direct the pipeline to autosplit the test dataset from the training dataset.
test_label_path = "gs://cloud-samples-data/ai-platform/embedding/goog-10k-2024/r11/test.tsv"  # @param{type: "string"}
# @markdown ---
# @markdown `train_steps`: Optional. The number of steps to perform model tuning. Must be greater than 30.
train_steps = 1000  # @param {type: "number"}
# @markdown ---
# @markdown `batch_size`: Optional. The training batch size.
batch_size = 128  # @param {type: "number"}
# @markdown ---
# @markdown `base_model_version_id`: Optional. Use this to specify what text embedding model to tune. This must be a stable version, for example `text-embedding-004`. To learn more, see [Supported Models](https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-text-embeddings#supported-models).
base_model_version_id = "text-embedding-004"  # @param ["textembedding-gecko@003", "text-embedding-004", "textembedding-gecko-multilingual@001", "text-multilingual-embedding-002"]
# @markdown ---
# @markdown `task_type`: Optional. The task type expected to be used during inference. See the [embedding API reference](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/text-embeddings-api#request_body) to learn more.
task_type = "DEFAULT"  # @param ["DEFAULT", "RETRIEVAL_QUERY", "RETRIEVAL_DOCUMENT", "SEMANTIC_SIMILARITY", "CLASSIFICATION", "CLUSTERING", "QUESTION_ANSWERING", "FACT_VERIFICATION"]
# @markdown ---
# @markdown `learning_rate_multiplier`: Optional. A multiplier to apply to the recommended learning rate. To train with the recommended learning rate, use 1.0.
learning_rate_multiplier = 1.0  # @param {type: "number"}
# @markdown ---
# @markdown `output_dimensionality`: Optional. The desired embedding dimension of your tuned model. This is only supported for models `text-embedding-004` and `text-multilingual-embedding-002`. To produce a model with the maximum embeddings size (for all existing models, this is 768), use -1.
output_dimensionality = 768  # @param {type: "number"}

# Synchronously validate some edge cases that will cause async validation to fail.
if base_model_version_id not in ["text-embedding-004", "text-multilingual-embedding-002"]:
    if task_type in ["QUESTION_ANSWERING", "FACT_VERIFICATION"]:
        raise ValueError(f"task_type '{task_type}' is not valid for model '{base_model_version_id}'.")

    if output_dimensionality not in [-1, 768]:
        raise ValueError(f"Model '{base_model_version_id}' does not support the output_dimensionality parameter.")

base_model = TextEmbeddingModel.from_pretrained(base_model_version_id)
if "TUNING_JOB_ID" in locals() and TUNING_JOB_ID:
    filter = f'pipelineJobUserId="{TUNING_JOB_ID}"'
    tuning_job = next(iter(pipeline_jobs.PipelineJob.list(filter=filter)))
    print(
        f"Got an existing tuning job '{tuning_job.name}' (state: {tuning_job.state.name})."
    )
else:
    tuning_result = base_model.tune_model(
        task_type=task_type,
        corpus_data=corpus_path,
        queries_data=queries_path,
        training_data=train_label_path,
        validation_data=validation_label_path,
        test_data=test_label_path,
        batch_size=batch_size,
        train_steps=train_steps,
        tuned_model_location=REGION,
        learning_rate_multiplier=learning_rate_multiplier,
        output_dimensionality=output_dimensionality,
    )
    tuning_job = pipeline_jobs.PipelineJob.get(tuning_result.pipeline_job_name)
    print(
        f"Got a fresh tuning job '{tuning_job.name}' (state: {tuning_job.state.name})."
    )
    print(
        f"(OPTIONAL) Set 'TUNING_JOB_ID' to '{tuning_job.name}' when you want to resume this tuning session."
    )

### Deploy the Tuned Text-Embedding Model

You can deploy a tuned model to an endpoint, once a tuning job is completed. Deploying a model can be done in one of these ways:

* Calling `deploy_tuned_model` method on a tuning result object from `tune_model` method on a base model object.
* Calling class method `TextEmbeddingModel.deploy_tuned_model` with a tuned model resource name. You can retrieve from a tuning job the uploaded, tuned model's resource name. This resource name is a string that follows the pattern projects/{PROJECT_ID}/locations/{REGION}/models/{MODEL_ID}.

**IMPORTANT**: Deployment of tuned text-embedding models, being custom trained, requires the allocation of serving resources like machines and accelerators within your project. `deploy_tuned_model`, upon its initial call, creates a tuned model deployment onto a fresh serving resource. Upon subsequent calls, it simply retrieves the tuned model deployment from the existing serving resource.

See also [using tuned models](https://cloud.google.com/vertex-ai/generative-ai/docs/models/tune-embeddings#use-tuned-model).


In [ ]:
MACHINE_TYPE = "a2-highgpu-1g"  # @param {type: "string"}
ACCELERATOR = "NVIDIA_TESLA_A100"  # @param {type: "string"}
ACCELERATOR_COUNT = 1  # @param {type: "number"}

# CAVEAT: Colab disruptions may cause 'tuning_result' to be undefined.
running_interactively = not os.getenv("IS_TESTING")
if "tuning_job" not in locals() and running_interactively:
    message = "[Action Required] Run the preceding code cells to define 'tuning_job'."
    raise RuntimeError(message)
if "tuning_job" in locals() and running_interactively:
    if "tuning_result" in locals():
        model = tuning_result.deploy_tuned_model(
            machine_type=MACHINE_TYPE,
            accelerator=ACCELERATOR,
            accelerator_count=ACCELERATOR_COUNT,
        )
        print(f"Got deployed, tuned {model=}")
    else:
        tuning_job.wait()
        tasks = tuning_job.task_details
        upload_task = next(t for t in tasks if "uploader" in t.task_name)
        upload_metadata = dict(upload_task.execution.metadata)
        tuned_model_name = upload_metadata["output:model_resource_name"]
        model = TextEmbeddingModel.deploy_tuned_model(
            tuned_model_name=tuned_model_name,
            machine_type=MACHINE_TYPE,
            accelerator=ACCELERATOR,
            accelerator_count=ACCELERATOR_COUNT,
        )
        print(f"Got deployed, tuned model '{tuned_model_name}'.")

### Examine Quality Metrics from Tuning Job

The pipeline assesses the performance of both the base and tuned models during tuning by calculating NDCG@10 metrics using validation and test datasets. These metrics are available in the pipeline job's `metrics` artifact.

In [ ]:
import pandas as pd

if "tuning_job" in locals() and tuning_job.done():
    tasks = tuning_job.task_details
    eval_task = next(t for t in tasks if "evaluator" in t.task_name)
    metrics = dict(eval_task.outputs["metrics"].artifacts[0].metadata)
    metrics_df = pd.DataFrame.from_dict(
        {"metric": metrics.keys(), "value": metrics.values()}
    )
    display(metrics_df.sort_values(by="metric", ignore_index=True))

### Get Tuned Embeddings for Downstream Tasks

Get embeddings from your tuned model for downstream tasks.

In [ ]:
import pandas as pd

if "model" in locals():
    texts = ["banana muffins?"]  # @param {type:"raw"}
    titles = ["none"]  # @param {type:"raw"}
    embedding_inputs = [
        TextEmbeddingInput(text=text, task_type=task_type, title=title)
        for text, title in zip(texts, titles)
    ]
    tuned_embeddings = [
        pd.Series(e.values) for e in model.get_embeddings(embedding_inputs)
    ]
    display(tuned_embeddings)

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.